In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nepsa/NepSA.csv
/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec/pretrained.word2vec.wv.vectors.npy
/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec/pretrained.word2vec.trainables.syn1neg.npy
/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec/pretrained.word2vec
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors_vocab.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_ngrams_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.syn1neg.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_vocab_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_

In [2]:
from pathlib import Path
df= pd.read_csv("/kaggle/input/nepsa/NepSA.csv")
df_copy= df.copy()
df.info()

scratch_output = "./scratch/"
finetuned_output = "./finetune/"

Path(scratch_output).mkdir()
Path(finetuned_output).mkdir()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4707 non-null   object
 1   Target  4707 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.7+ KB


In [3]:
# Tokenizing using nltk and creating a corpus for models to train on

import nltk
from nltk.tokenize import word_tokenize

sentences= df_copy['Text'].apply(lambda x: word_tokenize(x)).tolist()

In [4]:
from gensim.models import FastText

scratch_fasttext= FastText(sentences, vector_size= 300,sg=1 ,negative=10 ,window= 15, min_count=3,alpha=0.0025, epochs=200, workers=4, seed= 42)

In [5]:
print(f"दलाल: {scratch_fasttext.wv.most_similar('दलाल')}")
print(f"मुजि: {scratch_fasttext.wv.most_similar('मुजि')}")
print(f"रन्डि: {scratch_fasttext.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {scratch_fasttext.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {scratch_fasttext.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {scratch_fasttext.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {scratch_fasttext.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {scratch_fasttext.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलालि', 0.8507505059242249), ('दलाली', 0.8405889868736267), ('दले', 0.7199591994285583), ('गोरू', 0.6841635704040527), ('घुसिया', 0.6700232028961182), ('भारतिय', 0.669421374797821), ('थुईक्क', 0.666473388671875), ('पापी', 0.654083788394928), ('संसद', 0.6519933938980103), ('भुसिया', 0.6435674428939819)]
मुजि: [('मुखै', 0.733791708946228), ('मुर्ख', 0.7313032746315002), ('मुख्', 0.7287921905517578), ('मुत', 0.7127673625946045), ('मुर्दाबाद', 0.7093185782432556), ('चम्चा', 0.7061370611190796), ('तनाव', 0.705727756023407), ('मुर्दा', 0.7045921683311462), ('आय', 0.6999744176864624), ('मुजी', 0.6940966844558716)]
रन्डि: [('रन्दि', 0.8976467847824097), ('रन्डी', 0.8783191442489624), ('कन्डम', 0.8125463128089905), ('तेइ', 0.8120995163917542), ('थोक', 0.8053849339485168), ('छोरा', 0.8040271401405334), ('राडी', 0.8021667003631592), ('जाठा', 0.7988181114196777), ('सिदै', 0.7897745966911316), ('छोरो', 0.7896008491516113)]
चिक्ने: [('माचिक्ने', 0.8801254630088806), ('पढ्ने', 0.815369427204

In [6]:
print("Vectors shape:", scratch_fasttext.wv.vectors.shape)


Vectors shape: (2660, 300)


In [7]:
_ = scratch_fasttext.wv.vectors
_ = scratch_fasttext.wv.vectors_ngrams


scratch_fasttext.save(os.path.join(scratch_output,'scratch'))

In [8]:

pretrained_fasttext_path = "/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text"
pretrained = FastText.load(pretrained_fasttext_path)

In [9]:
print(f"दलाल: {pretrained.wv.most_similar('दलाल')}")
print(f"मुजि: {pretrained.wv.most_similar('मुजि')}")
print(f"रन्डि: {pretrained.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {pretrained.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {pretrained.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {pretrained.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {pretrained.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {pretrained.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलालहर', 0.7440781593322754), ('दलाली', 0.6854984760284424), ('विचौल', 0.6209348440170288), ('बिचौल', 0.6050319075584412), ('दलालबाट', 0.5906900763511658), ('दलालमार्फत', 0.5906495451927185), ('दलालपुँज', 0.5692156553268433), ('पुँजीपति', 0.5689678192138672), ('ोकरशाह', 0.5642825961112976), ('पूँजीपति', 0.5638065338134766)]
मुजि: [('मुजिर', 0.8002251982688904), ('मुजिबर', 0.7488963603973389), ('मुजिवर', 0.7370174527168274), ('मुजिब', 0.7313560843467712), ('मुजिव', 0.7237745523452759), ('मुजिबुल', 0.7164750099182129), ('मुजिम', 0.6959578394889832), ('मुजिवुल', 0.6847776174545288), ('मुजिङ', 0.6725438237190247), ('मुजीव', 0.6572784781455994)]
रन्डि: [('ब्रेन्डि', 0.6358718872070312), ('भुन्डि', 0.6007540225982666), ('एन्डि', 0.5524656176567078), ('ट्ेरन्डिङ', 0.5476298928260803), ('इन्डि', 0.5357617139816284), ('झण्डि', 0.5149215459823608), ('ट्रन्डिङ', 0.5131903290748596), ('झुन्डि', 0.5115798115730286), ('ट्रेन्डिङ', 0.5075634121894836), ('झुण्डि', 0.5072340965270996)]
चिक्ने:

In [10]:
print(pretrained.corpus_count)
print(pretrained.epochs)

17975484
50


# Finetuning
Following Resume training for Word2Vec from official Documentation, we apply the same to Fasttext

In [11]:
finetuned = pretrained
finetuned.build_vocab(sentences, update=True)

print(finetuned.corpus_count)
print(finetuned.epochs)

4707
50


In [12]:
finetuned.train(sentences, total_examples=finetuned.corpus_count ,epochs=200)

(5243402, 9137400)

In [13]:
print(f"दलाल: {finetuned.wv.most_similar('दलाल')}")
print(f"मुजि: {finetuned.wv.most_similar('मुजि')}")
print(f"रन्डि: {finetuned.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {finetuned.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {finetuned.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {finetuned.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {finetuned.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {finetuned.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलालहर', 0.6962102055549622), ('वदलाल', 0.6183063387870789), ('दलाली', 0.6166598796844482), ('दलालबाट', 0.616618812084198), ('दलालहरुल', 0.6100828647613525), ('दलालीहर', 0.5980662703514099), ('दला', 0.5905641913414001), ('लाल', 0.5859220027923584), ('दलालीप', 0.5828729271888733), ('दलालसंग', 0.5791537761688232)]
मुजि: [('मुजिर', 0.7511714100837708), ('मुजिबर', 0.7211726307868958), ('मुजिङ', 0.7162096500396729), ('मुजिम', 0.6950715780258179), ('मुजिब', 0.6893698573112488), ('मुजिवर', 0.6596695184707642), ('मुजिव', 0.6521907448768616), ('मुजिबुल', 0.6481915712356567), ('मुजिवुल', 0.626686155796051), ('मुजिकल', 0.599871814250946)]
रन्डि: [('भुन्डि', 0.700746476650238), ('रन्ड', 0.6753078699111938), ('इन्डि', 0.661927342414856), ('रन्डी', 0.658280074596405), ('झुन्डि', 0.643549382686615), ('ब्रेन्डि', 0.6343564987182617), ('एन्डि', 0.6242436170578003), ('झण्डि', 0.6150180101394653), ('झुण्डि', 0.5903967022895813), ('रन्डाल', 0.587599515914917)]
चिक्ने: [('माचिक्ने', 0.795196712017

In [14]:
_ = finetuned.wv.vectors
_ = finetuned.wv.vectors_ngrams


finetuned.save(os.path.join(finetuned_output,'finetune'))